In [46]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import math
import operator


In [47]:
# read saved prices
prices = pd.read_csv("./prices.csv")

In [48]:
print(requests.get('https://www.czc.cz/robots.txt').text)

User-agent: Sogou
User-agent: Baiduspider
User-agent: YoudaoBot
Disallow: /

Host: www.czc.cz

User-Agent: *
Disallow: /login
Disallow: /*/login
Disallow: /kosik/
Disallow: /dokoncit-registraci
Disallow: /email/*
Disallow: /change-password
Disallow: /zmenit-heslo/
Disallow: /*/nova$
Disallow: /*/novy$
Disallow: /*/priloha
Disallow: /*/upload-image
Disallow: /*?q-count=
Disallow: /*&q-count=
Disallow: /*?q-order=
Disallow: /*&q-order=
Disallow: /*?q-direction=
Disallow: /*&q-direction=
Disallow: /*?position=
Disallow: /*&position=
Disallow: /*?q-c*=
Disallow: /*&q-c*=
Disallow: /*?related=
Disallow: /*&related=
Disallow: /*.jsm
Disallow: /letaky
Disallow: /support
Disallow: /czc/products.nsf/
Disallow: /czc/media.nsf/
Disallow: /art_list.jsp
Disallow: /captcha.jsp
Disallow: /MAIN.aspx
Disallow: /Main.aspx
Disallow: /DEFAULT.aspx

Disallow: /*/kontakty
Disallow: /*/soukromi
Disallow: /*/pobocky
Disallow: /*/pro-firmy
Disallow: /*/volna-mista
Disallow: /*/autodiscover.xml

Allow: /geek/*/

In [49]:
prices.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,shop,timestamp,product,price
0,0,0,0,0,0.0,CZC,2019-12-16 17:44:28.432887,Mys,12
1,1,1,1,1,1.0,CZC,2019-12-16 17:44:28.446934,Notebook,122
2,2,2,2,2,2.0,CZC,2019-12-16 17:44:28.458476,Mobil,45
3,3,3,3,3,3.0,Alza,2019-12-16 17:44:28.470659,Mys,45
4,4,4,4,4,4.0,Alza,2019-12-16 17:44:28.481968,Notebook,150


In [50]:
class Downloader:
    
    def is_new_product_price(self, product): # checked if the price is different or not
        
        latest_price = 0
        
        for index, row in prices.iterrows():
            if(row["shop"] == self.name and row["product"] == product.get("name")):
                latest_price = row["price"]

        if(latest_price != product.get("price")):
            return True
        
    def update_prices(self): # if the price is different, add a new row
        products = self.get_products()
        
        for product in products:
            if(self.is_new_product_price(product)):
                print("added row")
                
             # how to add a row
                global prices
                prices = prices.append({"shop":self.name, "timestamp": datetime.now(), "product": product.get("name"), "price": product.get("price")}, ignore_index= True)
                
        
class Downloader_CZC(Downloader):
    
    name = "CZC"


class Downloader_Alza(Downloader):
    
    name = "Alza"
    

    def get_products(self):
        
        # stahnout html
        # pomoci BeatifulSoup nebo RegExp ziskat stranky a produkty
        # ziskat z nich cenu
        # vratit jako nejaky datovy typ ne nutne tento ale pak je potrab upravit metodu update_prices
        return [
            {"name":"Mys", "price": 45},
            {"name":"Notebook", "price": 150},
            {"name":"Mobil", "price": 60}
        ]
    
# pridat Downloader_Mall

In [51]:
# pridat Mall
downloaders = [Downloader_CZC(), Downloader_Alza()]

In [54]:
# Pro kazdy downloader stahne ceny, podiva se jestli se zmenily a pripadne je prida cenu do prices

for downloader in downloaders:
    downloader.update_prices()
    

AttributeError: 'Downloader_CZC' object has no attribute 'get_products'

In [43]:
prices.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,shop,timestamp,product,price
0,0,0,0,0.0,CZC,2019-12-16 17:44:28.432887,Mys,12
1,1,1,1,1.0,CZC,2019-12-16 17:44:28.446934,Notebook,122
2,2,2,2,2.0,CZC,2019-12-16 17:44:28.458476,Mobil,45
3,3,3,3,3.0,Alza,2019-12-16 17:44:28.470659,Mys,45
4,4,4,4,4.0,Alza,2019-12-16 17:44:28.481968,Notebook,150


In [41]:
# opravit vytvareni noveho sloupce na zacatku
prices.to_csv("prices.csv")

In [42]:
# zalohovani?
prices.to_csv("prices_backup_{}.csv".format(datetime.now()))

OSError: [Errno 22] Invalid argument: 'prices_backup_2020-01-05 17:07:20.124777.csv'